# HSN Classifier with OpenAI Function Calling

This notebook:
1. Loads the `HSN_SAC.xlsx` dataset.
2. Preprocesses HSN codes and descriptions.
3. Trains a dual-input Keras model.
4. Defines a `hsn_classifier` function.
5. Demonstrates OpenAI function-calling to invoke the classifier.

**Prerequisites**:
- Storing `HSN_SAC.xlsx` in the same directory.
- Setting environment variable `OPENAI_API_KEY` to our OpenAI API key.


In [ ]:
# Installing dependencies
!pip install --quiet pandas numpy scikit-learn tensorflow openai

You should consider upgrading via the 'C:\Users\Sunny\venvs\hsn_tf_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [19]:
# Necessary Imports and environment setup
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import pickle
import openai   

# Reading the key from the environment and assign it
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

print("✅ Successfully loaded OPENAI_API_KEY (first 4 chars):", openai.api_key[:10])

# ──────────────────────────────────────────


✅ Successfully loaded OPENAI_API_KEY (first 4 chars): sk-proj-TC


In [ ]:
# Loading and inspecting the dataset
import openpyxl  
import pandas as pd

sd = pd.read_excel(r"C:\Users\Sunny\venvs\hsn_tf_env\New folder (2)\HSN_SAC.xlsx")
sd.head()

,\nHSNCode,Description
0,01,LIVE ANIMALS
1,0101,"LIVE HORSES, ASSES, MULES AND HINNIES."
2,01011010,"LIVE HORSES, ASSES, MULES AND HINNIES PURE-BRE..."
3,01011020,"LIVE HORSES, ASSES, MULESANDHINNIES PURE-BRED ..."
4,01011090,"LIVE HORSES, ASSES, MULES AND HINNIES PURE-BRE..."


In [ ]:
# Stripping whitespace/newlines from all columns
sd.columns = sd.columns.str.strip()
print("Cleaned column names:", sd.columns.tolist())

Cleaned column names: ['HSNCode', 'Description']


In [23]:
# Printing out the raw columns so we can see exactly what they are
print("Raw column names in the spreadsheet:")
print(sd.columns.tolist())

# Renaming to standardized names: "raw_code" and "description"
sd = sd.rename(columns={
    "HSNCode": "raw_code",      
    "HSN Code": "raw_code",     
    "Description": "description" 
})

# If we see any other variation in the printed list, add it as another key in that dict.

print("\n After renaming to 'raw_code' and 'description':")
print(sd.columns.tolist())

Raw column names in the spreadsheet:
['HSNCode', 'Description']

 After renaming to 'raw_code' and 'description':
['raw_code', 'description']


In [24]:
# Encoding target (HSNCode) as categories
le = LabelEncoder()
sd["label"] = le.fit_transform(sd["raw_code"])  # using the exact name printed by sd.columns
num_classes = len(le.classes_)
print("Number of labels:", num_classes)

# Tokenizing HSN code (character-level)
code_tokenizer = Tokenizer(char_level=True, filters=None, lower=False)
code_tokenizer.fit_on_texts(sd["raw_code"].astype(str))
X_code_seq = code_tokenizer.texts_to_sequences(sd["raw_code"].astype(str))
MAX_CODE_LEN = max(len(x) for x in sd["raw_code"].astype(str))
X_code = pad_sequences(X_code_seq, maxlen=MAX_CODE_LEN, padding="post")

# Tokenize description (word-level)
DESC_VOCAB_SIZE = 10000
DESC_MAX_LEN = 50
desc_tokenizer = Tokenizer(num_words=DESC_VOCAB_SIZE, oov_token="<OOV>")
desc_tokenizer.fit_on_texts(sd["description"].astype(str))
X_desc_seq = desc_tokenizer.texts_to_sequences(sd["description"].astype(str))
X_desc = pad_sequences(X_desc_seq, maxlen=DESC_MAX_LEN, padding="post")

# Preparing target one-hot
y = tf.keras.utils.to_categorical(sd["label"], num_classes=num_classes)

# Splitting into train/validation
X_code_train, X_code_val, X_desc_train, X_desc_val, y_train, y_val = train_test_split(
    X_code, X_desc, y, test_size=0.2, random_state=42
)

print("Shapes:")
print(" X_code_train:", X_code_train.shape)
print(" X_desc_train:", X_desc_train.shape)
print(" y_train:", y_train.shape)

Number of labels: 21568
Shapes:
 X_code_train: (17254, 8)
 X_desc_train: (17254, 50)
 y_train: (17254, 21568)


In [ ]:
# Building the dual-input Keras model

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Conv1D, GlobalMaxPooling1D,
    concatenate, Dense, Dropout
)

# Code branch (character-level)
code_input = Input(shape=(MAX_CODE_LEN,), name="code_input")
code_vocab_size = len(code_tokenizer.word_index) + 1  # +1 for padding token
code_embedding_dim = 32  # arbitrary choice
code_emb = Embedding(
    input_dim=code_vocab_size,
    output_dim=code_embedding_dim,
    name="code_emb"
)(code_input)
code_lstm_units = 64  # arbitrary choice
code_lstm = LSTM(code_lstm_units, name="code_lstm")(code_emb)

# Description branch (word-level)
desc_input = Input(shape=(DESC_MAX_LEN,), name="desc_input")
DESC_VOCAB_SIZE = len(desc_tokenizer.word_index) + 1
desc_embedding_dim = 128  # arbitrary choice
desc_emb = Embedding(
    input_dim=DESC_VOCAB_SIZE,
    output_dim=desc_embedding_dim,
    name="desc_emb"
)(desc_input)
desc_conv_filters = 128  # arbitrary choice
desc_conv = Conv1D(
    filters=desc_conv_filters,
    kernel_size=5,         # arbitrary
    activation="relu",
    name="desc_conv"
)(desc_emb)
desc_pool = GlobalMaxPooling1D(name="desc_pool")(desc_conv)

# Merging and classification head
merged = concatenate([code_lstm, desc_pool], name="concat_layer")
dropout_rate = 0.4  # arbitrary choice
x = Dropout(dropout_rate, name="dropout1")(merged)
dense_units = 128  # arbitrary choice
x = Dense(dense_units, activation="relu", name="dense1")(x)
x = Dropout(dropout_rate, name="dropout2")(x)
output = Dense(num_classes, activation="softmax", name="output_layer")(x)

model = Model(inputs=[code_input, desc_input], outputs=output, name="HSN_Classifier")
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()


Model: "HSN_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ desc_input          │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ code_input          │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ desc_emb            │ (None, 50, 128)   │  1,227,648 │ desc_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ code_emb            │ (None, 8, 32)     │        384 │ code_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ desc_conv (Conv1D)  │ (None, 46, 128)   │     82,048 │ desc_emb[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ code_lstm (LSTM)    │ (None, 64)        │     24,832 │ code_emb[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ desc_pool           │ (None, 128)       │          0 │ desc_conv[0][0]   │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat_layer        │ (None, 192)       │          0 │ code_lstm[0][0],  │
│ (Concatenate)       │                   │            │ desc_pool[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout1 (Dropout)  │ (None, 192)       │          0 │ concat_layer[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense1 (Dense)      │ (None, 128)       │     24,704 │ dropout1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout2 (Dropout)  │ (None, 128)       │          0 │ dense1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 21568)     │  2,782,272 │ dropout2[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,141,888 (15.80 MB)

 Trainable params: 4,141,888 (15.80 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Training the model & saving artifacts

history = model.fit(
    {"code_input": X_code_train, "desc_input": X_desc_train},
    y_train,
    validation_data=(
        {"code_input": X_code_val, "desc_input": X_desc_val},
        y_val
    ),
    epochs=10,            
    batch_size=32,       # arbitrary batch size
    verbose=2
)

# Saving model + tokenizers + label encoder + constants
model.save("hsn_classifier_model.h5")

with open("code_tokenizer.pkl", "wb") as f:
    pickle.dump(code_tokenizer, f)
with open("desc_tokenizer.pkl", "wb") as f:
    pickle.dump(desc_tokenizer, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

# Saving the fixed constants
with open("constants.pkl", "wb") as f:
    pickle.dump({"MAX_CODE_LEN": MAX_CODE_LEN, "DESC_MAX_LEN": DESC_MAX_LEN}, f)


Epoch 1/10
540/540 - 78s - 144ms/step - accuracy: 1.7387e-04 - loss: 9.3342 - val_accuracy: 0.0000e+00 - val_loss: 11.9784
Epoch 2/10
540/540 - 30s - 56ms/step - accuracy: 6.9549e-04 - loss: 9.0824 - val_accuracy: 0.0000e+00 - val_loss: 12.3790
Epoch 3/10
540/540 - 29s - 55ms/step - accuracy: 0.0018 - loss: 8.7917 - val_accuracy: 0.0000e+00 - val_loss: 12.8206
Epoch 4/10
540/540 - 29s - 54ms/step - accuracy: 0.0039 - loss: 8.4573 - val_accuracy: 0.0000e+00 - val_loss: 13.1907
Epoch 5/10
540/540 - 29s - 54ms/step - accuracy: 0.0069 - loss: 8.0928 - val_accuracy: 0.0000e+00 - val_loss: 13.5888
Epoch 6/10
540/540 - 29s - 54ms/step - accuracy: 0.0118 - loss: 7.7192 - val_accuracy: 0.0000e+00 - val_loss: 13.9630
Epoch 7/10
540/540 - 29s - 54ms/step - accuracy: 0.0155 - loss: 7.3911 - val_accuracy: 0.0000e+00 - val_loss: 14.4529
Epoch 8/10
540/540 - 29s - 54ms/step - accuracy: 0.0234 - loss: 7.0701 - val_accuracy: 0.0000e+00 - val_loss: 14.8802
Epoch 9/10
540/540 - 29s - 54ms/step - accuracy

In [26]:
# Loading the model/tokenizers/constants 
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Loading the Keras model
try:
    _hsn_model = load_model("hsn_classifier_model.h5")
    print(" Model loaded successfully.")
except Exception as e:
    print(" Failed to load model:", e)

# Loading the code_tokenizer, desc_tokenizer, and label_encoder
try:
    with open("code_tokenizer.pkl", "rb") as f:
        _code_tokenizer = pickle.load(f)
    with open("desc_tokenizer.pkl", "rb") as f:
        _desc_tokenizer = pickle.load(f)
    with open("label_encoder.pkl", "rb") as f:
        _label_encoder = pickle.load(f)
    print(" Tokenizers and label encoder loaded successfully.")
except Exception as e:
    print(" Failed to load tokenizer/label encoder:", e)

# Loading padding constants
try:
    with open("constants.pkl", "rb") as f:
        consts = pickle.load(f)
    MAX_CODE_LEN = consts["MAX_CODE_LEN"]
    DESC_MAX_LEN = consts["DESC_MAX_LEN"]
    print(f" Constants loaded: MAX_CODE_LEN = {MAX_CODE_LEN}, DESC_MAX_LEN = {DESC_MAX_LEN}")
except Exception as e:
    print(" Failed to load constants.pkl:", e)

# Defining the inference function if not already defined 
def hsn_classifier(hsn_code: str, description: str) -> dict:
    
    # Preprocessing the code
    code_seq = _code_tokenizer.texts_to_sequences([hsn_code])
    code_input = pad_sequences(code_seq, maxlen=MAX_CODE_LEN, padding="post", truncating="post")

    # Preprocessing the description
    desc_seq = _desc_tokenizer.texts_to_sequences([description])
    desc_input = pad_sequences(desc_seq, maxlen=DESC_MAX_LEN, padding="post", truncating="post")

    # Predicting
    probs = _hsn_model.predict({"code_input": code_input, "desc_input": desc_input})
    idx = int(np.argmax(probs, axis=1)[0])
    predicted_code = _label_encoder.inverse_transform([idx])[0]
    confidence = float(probs[0][idx])

    return {
        "predicted_hsn_code": str(predicted_code),
        "confidence": confidence
    }

# Running a quick “sanity check” 
sample_code = "1006"   
sample_desc = "Rice, semi-milled or wholly milled, whether or not polished or glazed"

try:
    result = hsn_classifier(sample_code, sample_desc)
    print(" hsn_classifier output:", result)
except Exception as e:
    print(" hsn_classifier(…) raised an error:", e)


 Model loaded successfully.
 Tokenizers and label encoder loaded successfully.
 Constants loaded: MAX_CODE_LEN = 8, DESC_MAX_LEN = 50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
 hsn_classifier output: {'predicted_hsn_code': '8521', 'confidence': 0.006479457952082157}


In [27]:
import os
import openai
from openai import AuthenticationError, RateLimitError, OpenAIError

# (Optional) confirming our key is loaded:
print("OPENAI_API_KEY starts with:", os.getenv("OPENAI_API_KEY")[:10], "…")

# A simple chat call, catching the proper exceptions:
try:
    resp = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role":"user","content":"Hello, GPT!"}],
        
    )
    print(" GPT replied:", resp.choices[0].message.content)
except AuthenticationError as ae:
    print(" AuthenticationError—your API key might be invalid:", ae)
except RateLimitError as re:
    print(" RateLimitError—quota exceeded or too many requests:", re)
except OpenAIError as oe:
    print(" Other OpenAI API error:", oe)
except Exception as e:
    print(" Some other exception occurred:", e)


OPENAI_API_KEY starts with: sk-proj-TC …
 GPT replied: Hello! How can I assist you today?


In [29]:
import json
import time
import openai
from openai import AuthenticationError, RateLimitError, OpenAIError

# Defining our function schema exactly as before
function_spec = {
    "name": "hsn_classifier",
    "description": "Given an HSN code and description, return the predicted 8-digit HSN code and confidence.",
    "parameters": {
        "type": "object",
        "properties": {
            "hsn_code": {
                "type": "string",
                "description": "The raw HSN code to validate"
            },
            "description": {
                "type": "string",
                "description": "The item’s description in plain English"
            }
        },
        "required": ["hsn_code", "description"]
    }
}


# A small retry wrapper to handle transient 429s 
def safe_chat_completion(**kwargs):
    for attempt in range(3):
        try:
            return openai.chat.completions.create(**kwargs)
        except RateLimitError:
            print(f"[Attempt {attempt+1}] RateLimitError — sleeping 1s…")
            time.sleep(1)
    raise RuntimeError("RateLimitError persisted after 3 retries.")


# The “run_hsn_agent” function:
def run_hsn_agent(user_code: str, user_desc: str) -> str:
    
# Asking GPT whether to call our function
    try:
        initial_resp = safe_chat_completion(
            model="gpt-4o",  # use a model you have confirmed works
            messages=[
                {
                    "role": "user",
                    "content": (
                        f"I have HSN code '{user_code}' and description '{user_desc}'. "
                        "Please verify or correct the HSN code and tell me how confident you are."
                    )
                }
            ],
            functions=[function_spec],
            function_call="auto"
        )
    except AuthenticationError as ae:
        raise RuntimeError(" AuthenticationError: invalid or missing API key") from ae
    except RateLimitError as re:
        raise RuntimeError(" RateLimitError: no remaining quota or too many requests") from re
    except OpenAIError as oe:
        raise RuntimeError(" Other OpenAI API error") from oe

    # Grabbing the single message that the model returned
    message = initial_resp.choices[0].message

    # Printing the content & function_call (if any)
    print("\n GPT’s first reply (with function_call or direct content):")
    print(json.dumps({
        "role": message.role,
        # message.function_call is either None or an object with .name and .arguments
        "function_call": ( 
            {
                "name": message.function_call.name,
                "arguments": message.function_call.arguments
            } 
            if message.function_call is not None else None
        ),
        "content": message.content
    }, indent=2))

    # If GPT requested a function call…
    if message.function_call is not None:
        func_name = message.function_call.name
        # arguments is a JSON‐encoded string, so load it
        func_args = json.loads(message.function_call.arguments)
        print(f"\n GPT asked to call: {func_name} with args: {func_args}")

        if func_name == "hsn_classifier":
            # Run your local inference function
            func_response = hsn_classifier(
                hsn_code=func_args.get("hsn_code", ""),
                description=func_args.get("description", "")
            )
            print(" Local hsn_classifier returned:", func_response)
        else:
            raise RuntimeError(f"Unexpected function name: {func_name}")

        # Sending the function’s output back to GPT
        second_resp = safe_chat_completion(
            model="gpt-4o",
            messages=[
                # Re‐sending the original user message (optional but recommended)
                {"role": "user", "content": f"I asked about HSN '{user_code}' and '{user_desc}'."},
                # The assistant’s request to call the function
                message,
                # Our function’s response as a “function” role
                {
                    "role": "function",
                    "name": func_name,
                    "content": json.dumps(func_response)
                }
            ]
        )

        final_msg = second_resp.choices[0].message
        print("\nFinal assistant reply (after function call):")
        print(final_msg.content)
        return final_msg.content

    else:
        # If GPT replied directly without calling the function
        print("\n GPT replied directly (no function call).")
        print(message.content)
        return message.content


#Finally, run the HSN agent on a sample code + description:

sample_code = "1006.30"
sample_desc = "Rice, semi-milled or wholly milled, whether or not polished or glazed"

print("\n Running HSN agent with sample input…")
agent_reply = run_hsn_agent(sample_code, sample_desc)
print("\n→ Agent’s final response:", agent_reply)



 Running HSN agent with sample input…

 GPT’s first reply (with function_call or direct content):
{
  "role": "assistant",
  "function_call": {
    "name": "hsn_classifier",
    "arguments": "{\"hsn_code\":\"1006.30\",\"description\":\"Rice, semi-milled or wholly milled, whether or not polished or glazed\"}"
  },
  "content": null
}

 GPT asked to call: hsn_classifier with args: {'hsn_code': '1006.30', 'description': 'Rice, semi-milled or wholly milled, whether or not polished or glazed'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
 Local hsn_classifier returned: {'predicted_hsn_code': '61130000', 'confidence': 0.003788026748225093}

Final assistant reply (after function call):
The HSN code '1006.30' refers to "Rice, semi-milled or wholly milled, whether or not polished or glazed." This categorization is used in trade to classify rice that has been processed to remove some or all of the outer bran layer and may also be polished or glazed to improve its appearance and cooking qualities. Let 